In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Algoritmo di Shor

*Stima di utilizzo: Tre secondi su un processore Eagle r3 (NOTA: Questa è solo una stima. Il vostro tempo di esecuzione potrebbe variare.)*

[L'algoritmo di Shor,](https://epubs.siam.org/doi/abs/10.1137/S0036144598347011) sviluppato da Peter Shor nel 1994, è un algoritmo quantistico rivoluzionario per la fattorizzazione di interi in tempo polinomiale. La sua importanza risiede nella capacità di fattorizzare grandi interi in modo esponenzialmente più veloce rispetto a qualsiasi algoritmo classico conosciuto, minacciando la sicurezza di sistemi crittografici ampiamente utilizzati come RSA, che si basano sulla difficoltà di fattorizzare numeri grandi. Risolvendo efficacemente questo problema su un computer quantistico sufficientemente potente, l'algoritmo di Shor potrebbe rivoluzionare campi come la crittografia, la sicurezza informatica e la matematica computazionale, sottolineando il potere trasformativo del calcolo quantistico.

Questo tutorial si concentra sulla dimostrazione dell'algoritmo di Shor fattorizzando 15 su un computer quantistico.

Prima definiamo il problema della ricerca dell'ordine e costruiamo i circuiti corrispondenti dal protocollo di stima della fase quantistica. Successivamente, eseguiamo i circuiti di ricerca dell'ordine su hardware reale utilizzando i circuiti di profondità minima che possiamo traspilare. L'ultima sezione completa l'algoritmo di Shor collegando il problema della ricerca dell'ordine alla fattorizzazione di interi.

Concludiamo il tutorial con una discussione su altre dimostrazioni dell'algoritmo di Shor su hardware reale, concentrandoci sia su implementazioni generiche sia su quelle adattate per fattorizzare interi specifici come 15 e 21.
Nota: Questo tutorial si concentra maggiormente sull'implementazione e la dimostrazione dei circuiti riguardanti l'algoritmo di Shor. Per una risorsa educativa approfondita sul materiale, si prega di fare riferimento al corso [Fundamentals of quantum algorithms](/learning/courses/fundamentals-of-quantum-algorithms/phase-estimation-and-factoring/introduction) del Dr. John Watrous, e agli articoli nella sezione [Riferimenti](#references).
### Requisiti
Prima di iniziare questo tutorial, assicuratevi di avere installato quanto segue:
- Qiskit SDK v2.0 o successivo, con supporto per la [visualizzazione](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.40 o successivo (`pip install qiskit-ibm-runtime`)
### Configurazione

In [ ]:
import numpy as np
import pandas as pd
from fractions import Fraction
from math import floor, gcd, log

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT, UnitaryGate
from qiskit.transpiler import CouplingMap, generate_preset_pass_manager
from qiskit.visualization import plot_histogram

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler

## Passo 1: Mappare gli input classici a un problema quantistico
### Contesto
L'algoritmo di Shor per la fattorizzazione di interi utilizza un problema intermediario noto come problema della *ricerca dell'ordine*. In questa sezione, dimostriamo come risolvere il problema della ricerca dell'ordine utilizzando la *stima della fase quantistica*.
### Problema della stima della fase
Nel problema della stima della fase, ci viene dato uno stato quantistico $\ket{\psi}$ di $n$ qubit, insieme a un circuito quantistico unitario che agisce su $n$ qubit. Ci viene promesso che $\ket{\psi}$ è un autovettore della matrice unitaria $U$ che descrive l'azione del circuito, e il nostro obiettivo è calcolare o approssimare l'autovalore $\lambda = e^{2 \pi i \theta}$ a cui $\ket{\psi}$ corrisponde. In altre parole, il circuito dovrebbe restituire un'approssimazione del numero $\theta \in [0, 1)$ che soddisfa $$U \ket{\psi}= e^{2 \pi i \theta} \ket{\psi}.$$
L'obiettivo del circuito di stima della fase è approssimare $\theta$ in $m$ bit. Matematicamente parlando, vorremmo trovare $y$ tale che $\theta \approx y / 2^m$, dove $y \in {0, 1, 2, \dots, 2^{m-1}}$. L'immagine seguente mostra il circuito quantistico che stima $y$ in $m$ bit effettuando una misura su $m$ qubit.
![Quantum phase estimation circuit](../learning/images/courses/fundamentals-of-quantum-algorithms/phase-estimation-and-factoring/phase-estimation-procedure.svg)
Nel circuito sopra, i $m$ qubit superiori sono inizializzati nello stato $\ket{0^m}$, e gli $n$ qubit inferiori sono inizializzati in $\ket{\psi}$, che è promesso essere un autovettore di $U$. Il primo ingrediente nel circuito di stima della fase sono le operazioni unitarie controllate che sono responsabili di eseguire un *phase kickback* al loro qubit di controllo corrispondente. Queste unitarie controllate sono esponenziate in accordo alla posizione del qubit di controllo, variando dal bit meno significativo al bit più significativo. Poiché $\ket{\psi}$ è un autovettore di $U$, lo stato degli $n$ qubit inferiori non è influenzato da questa operazione, ma l'informazione di fase dell'autovalore si propaga ai $m$ qubit superiori.
Si scopre che dopo l'operazione di phase kickback tramite unitarie controllate, tutti i possibili stati dei $m$ qubit superiori sono ortonormali tra loro per ogni autovettore $\ket{\psi}$ dell'unitaria $U$. Pertanto, questi stati sono perfettamente distinguibili, e possiamo ruotare la base che formano indietro alla base computazionale per effettuare una misura. Un'analisi matematica mostra che questa matrice di rotazione corrisponde alla trasformata di Fourier quantistica (QFT) inversa nello spazio di Hilbert a $2^m$ dimensioni. L'intuizione dietro questo è che la struttura periodica degli operatori di esponenziazione modulare è codificata nello stato quantistico, e la QFT converte questa periodicità in picchi misurabili nel dominio delle frequenze.

Per una comprensione più approfondita del motivo per cui il circuito QFT è impiegato nell'algoritmo di Shor, rimandiamo il lettore al corso [Fundamentals of quantum algorithms](/learning/courses/fundamentals-of-quantum-algorithms/phase-estimation-and-factoring/introduction).
Siamo ora pronti a utilizzare il circuito di stima della fase per la ricerca dell'ordine.
### Problema della ricerca dell'ordine
Per definire il problema della ricerca dell'ordine, iniziamo con alcuni concetti di teoria dei numeri. Prima di tutto, per qualsiasi intero positivo dato $N$, definiamo l'insieme $\mathbb{Z}_N$ come $$\mathbb{Z}_N = {0, 1, 2, \dots, N-1}.$$
Tutte le operazioni aritmetiche in $\mathbb{Z}_N$ sono eseguite modulo $N$. In particolare, tutti gli elementi $a \in \mathbb{Z}_n$ che sono coprimi con $N$ sono speciali e costituiscono $\mathbb{Z}^*_N$ come $$\mathbb{Z}^*_N = { a \in \mathbb{Z}_N : \mathrm{gcd}(a, N)=1 }.$$
Per un elemento $a \in \mathbb{Z}^*_N$, il più piccolo intero positivo $r$ tale che $$a^r \equiv 1 \; (\mathrm{mod} \; N)$$ è definito come l'*ordine* di $a$ modulo $N$. Come vedremo più avanti, trovare l'ordine di un $a \in \mathbb{Z}^*_N$ ci permetterà di fattorizzare $N$.
Per costruire il circuito di ricerca dell'ordine dal circuito di stima della fase, abbiamo bisogno di due considerazioni. Prima, dobbiamo definire l'unitaria $U$ che ci permetterà di trovare l'ordine $r$, e secondo, dobbiamo definire un autovettore $\ket{\psi}$ di $U$ per preparare lo stato iniziale del circuito di stima della fase.

Per collegare il problema della ricerca dell'ordine alla stima della fase, consideriamo l'operazione definita su un sistema i cui stati classici corrispondono a $\mathbb{Z}_N$, dove moltiplichiamo per un elemento fisso $a \in \mathbb{Z}^*_N$. In particolare, definiamo questo operatore di moltiplicazione $M_a$ tale che $$M_a \ket{x} = \ket{ax \; (\mathrm{mod} \; N)}$$ per ogni $x \in \mathbb{Z}_N$. Notate che è implicito che stiamo prendendo il prodotto modulo $N$ all'interno del ket sul lato destro dell'equazione. Un'analisi matematica mostra che $M_a$ è un operatore unitario. Inoltre, si scopre che $M_a$ ha coppie autovettore-autovalore che ci permettono di collegare l'ordine $r$ di $a$ al problema della stima della fase. Specificamente, per qualsiasi scelta di $j \in {0, \dots, r-1}$, abbiamo che $$\ket{\psi_j} = \frac{1}{\sqrt{r}} \sum^{r-1}_{k=0} \omega^{-jk}_{r} \ket{a^k}$$ è un autovettore di $M_a$ il cui autovalore corrispondente è $\omega^{j}_{r}$, dove $$\omega^{j}_{r} = e^{2 \pi i \frac{j}{r}}.$$
Per osservazione, vediamo che una coppia autovettore/autovalore conveniente è lo stato $\ket{\psi_1}$ con $\omega^{1}_{r} = e^{2 \pi i \frac{1}{r}}$. Pertanto, se potessimo trovare l'autovettore $\ket{\psi_1}$, potremmo stimare la fase $\theta=1/r$ con il nostro circuito quantistico e quindi ottenere una stima dell'ordine $r$. Tuttavia, non è facile farlo, e dobbiamo considerare un'alternativa.

Consideriamo cosa risulterebbe dal circuito se preparassimo lo stato computazionale $\ket{1}$ come stato iniziale. Questo non è un autostato di $M_a$, ma è la sovrapposizione uniforme degli autostati che abbiamo appena descritto sopra. In altre parole, vale la seguente relazione. $$ \ket{1} = \frac{1}{\sqrt{r}} \sum^{r-1}_{k=0} \ket{\psi_k} $$
L'implicazione dell'equazione sopra è che se impostiamo lo stato iniziale a $\ket{1}$, otterremo precisamente lo stesso risultato di misura come se avessimo scelto $k \in { 0, \dots, r-1}$ uniformemente a caso e usato $\ket{\psi_k}$ come autovettore nel circuito di stima della fase. In altre parole, una misura dei $m$ qubit superiori produce un'approssimazione $y / 2^m$ al valore $k / r$ dove $k \in { 0, \dots, r-1}$ è scelto uniformemente a caso. Questo ci permette di apprendere $r$ con un alto grado di confidenza dopo diverse esecuzioni indipendenti, che era il nostro obiettivo.
### Operatori di esponenziazione modulare
Finora, abbiamo collegato il problema della stima della fase al problema della ricerca dell'ordine definendo $U = M_a$ e $\ket{\psi} = \ket{1}$ nel nostro circuito quantistico. Pertanto, l'ultimo ingrediente rimanente è trovare un modo efficiente per definire esponenziali modulari di $M_a$ come $M_a^k$ per $k = 1, 2, 4, \dots, 2^{m-1}$.
Per eseguire questo calcolo, scopriamo che per qualsiasi potenza $k$ scegliamo, possiamo creare un circuito per $M_a^k$ non iterando $k$ volte il circuito per $M_a$, ma invece calcolando $b = a^k \; \mathrm{mod} \; N$ e poi usando il circuito per $M_b$. Poiché abbiamo bisogno solo delle potenze che sono esse stesse potenze di 2, possiamo farlo in modo classicamente efficiente usando l'elevamento al quadrato iterativo.
## Passo 2: Ottimizzare il problema per l'esecuzione su hardware quantistico
### Esempio specifico con $N = 15$ e $a=2$
Possiamo fare una pausa qui per discutere un esempio specifico e costruire il circuito di ricerca dell'ordine per $N=15$. Notate che i possibili $a \in \mathbb{Z}_N^*$ non banali per $N=15$ sono $a \in {2, 4, 7, 8, 11, 13, 14 }$. Per questo esempio, scegliamo $a=2$. Costruiremo l'operatore $M_2$ e gli operatori di esponenziazione modulare $M_2^k$.
L'azione di $M_2$ sugli stati della base computazionale è la seguente.
$$M_2 \ket{0} = \ket{0} \quad M_2 \ket{5} = \ket{10} \quad M_2 \ket{10} = \ket{5}$$
$$M_2 \ket{1} = \ket{2} \quad M_2 \ket{6} = \ket{12} \quad M_2 \ket{11} = \ket{7}$$
$$M_2 \ket{2} = \ket{4} \quad M_2 \ket{7} = \ket{14} \quad M_2 \ket{12} = \ket{9}$$
$$M_2 \ket{3} = \ket{6} \quad M_2 \ket{8} = \ket{1} \quad M_2 \ket{13} = \ket{11}$$
$$M_2 \ket{4} = \ket{8} \quad M_2 \ket{9} = \ket{3} \quad M_2 \ket{14} = \ket{13}$$
Per osservazione, possiamo vedere che gli stati di base sono mescolati, quindi abbiamo una matrice di permutazione. Possiamo costruire questa operazione su quattro qubit con porte swap. Di seguito, costruiamo le operazioni $M_2$ e $M_2$ controllata.

In [2]:
def M2mod15():
    """
    M2 (mod 15)
    """
    b = 2
    U = QuantumCircuit(4)

    U.swap(2, 3)
    U.swap(1, 2)
    U.swap(0, 1)

    U = U.to_gate()
    U.name = f"M_{b}"

    return U

In [3]:
# Get the M2 operator
M2 = M2mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(4)
circ.compose(M2, inplace=True)
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0a8885f1-91d4-40bd-912d-dc5eea05f5bd-0.avif" alt="Output of the previous code cell" />

In [4]:
def controlled_M2mod15():
    """
    Controlled M2 (mod 15)
    """
    b = 2
    U = QuantumCircuit(4)

    U.swap(2, 3)
    U.swap(1, 2)
    U.swap(0, 1)

    U = U.to_gate()
    U.name = f"M_{b}"
    c_U = U.control()

    return c_U

In [5]:
# Get the controlled-M2 operator
controlled_M2 = controlled_M2mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(5)
circ.compose(controlled_M2, inplace=True)
circ.decompose(reps=1).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/ab7fe331-2f9e-47ca-ba3b-f5d67992062a-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/ab7fe331-2f9e-47ca-ba3b-f5d67992062a-0.avif)

Le porte che agiscono su più di due qubit saranno ulteriormente decomposte in porte a due qubit.

In [6]:
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/13b4841d-a4ac-46bd-b4d0-d111b3017189-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/13b4841d-a4ac-46bd-b4d0-d111b3017189-0.avif)

Ora dobbiamo costruire gli operatori di esponenziazione modulare. Per ottenere una precisione sufficiente nella stima della fase, utilizzeremo otto qubit per la misura di stima. Pertanto, dobbiamo costruire $M_b$ con $b = a^{2^k} \; (\mathrm{mod} \; N)$ per ogni $k = 0, 1, \dots, 7$.

In [7]:
def a2kmodN(a, k, N):
    """Compute a^{2^k} (mod N) by repeated squaring"""
    for _ in range(k):
        a = int(np.mod(a**2, N))
    return a

In [8]:
k_list = range(8)
b_list = [a2kmodN(2, k, 15) for k in k_list]

print(b_list)

[2, 4, 1, 1, 1, 1, 1, 1]


As we can see from the list of $b$ values, in addition to $M_2$ that we previously constructed, we also need to build $M_4$ and $M_1$. Note that $M_1$ acts trivially on the computational basis states, so it is simply the identity operator.

$M_4$ acts on the computational basis states as follows.
$$M_4 \ket{0} = \ket{0} \quad M_4 \ket{5} = \ket{5} \quad M_4 \ket{10} = \ket{10}$$
$$M_4 \ket{1} = \ket{4} \quad M_4 \ket{6} = \ket{9} \quad M_4 \ket{11} = \ket{14}$$
$$M_4 \ket{2} = \ket{8} \quad M_4 \ket{7} = \ket{13} \quad M_4 \ket{12} = \ket{3}$$
$$M_4 \ket{3} = \ket{12} \quad M_4 \ket{8} = \ket{2} \quad M_4 \ket{13} = \ket{7}$$
$$M_4 \ket{4} = \ket{1} \quad M_4 \ket{9} = \ket{6} \quad M_4 \ket{14} = \ket{11}$$

Therefore, this permutation can be constructed with the following swap operation.

In [9]:
def M4mod15():
    """
    M4 (mod 15)
    """
    b = 4
    U = QuantumCircuit(4)

    U.swap(1, 3)
    U.swap(0, 2)

    U = U.to_gate()
    U.name = f"M_{b}"

    return U

In [10]:
# Get the M4 operator
M4 = M4mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(4)
circ.compose(M4, inplace=True)
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/be041e3d-28b1-453e-983e-184c2366aeb9-0.avif" alt="Output of the previous code cell" />

In [11]:
def controlled_M4mod15():
    """
    Controlled M4 (mod 15)
    """
    b = 4
    U = QuantumCircuit(4)

    U.swap(1, 3)
    U.swap(0, 2)

    U = U.to_gate()
    U.name = f"M_{b}"
    c_U = U.control()

    return c_U

In [12]:
# Get the controlled-M4 operator
controlled_M4 = controlled_M4mod15()

# Add it to a circuit and plot
circ = QuantumCircuit(5)
circ.compose(controlled_M4, inplace=True)
circ.decompose(reps=1).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/8d943b00-a502-4157-8a0d-13fb1f55e705-0.avif" alt="Output of the previous code cell" />

Gates acting on more than two qubits will be further decomposed into two-qubit gates.

In [13]:
circ.decompose(reps=2).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/68399eef-5e55-4c95-a8a4-c8efaebd34b9-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/8d943b00-a502-4157-8a0d-13fb1f55e705-0.avif)

Le porte che agiscono su più di due qubit saranno ulteriormente decomposte in porte a due qubit.

In [14]:
def mod_mult_gate(b, N):
    """
    Modular multiplication gate from permutation matrix.
    """
    if gcd(b, N) > 1:
        print(f"Error: gcd({b},{N}) > 1")
    else:
        n = floor(log(N - 1, 2)) + 1
        U = np.full((2**n, 2**n), 0)
        for x in range(N):
            U[b * x % N][x] = 1
        for x in range(N, 2**n):
            U[x][x] = 1
        G = UnitaryGate(U)
        G.name = f"M_{b}"
        return G

In [15]:
# Let's build M2 using the permutation matrix definition
M2_other = mod_mult_gate(2, 15)

# Add it to a circuit
circ = QuantumCircuit(4)
circ.compose(M2_other, inplace=True)
circ = circ.decompose()

# Transpile the circuit and get the depth
coupling_map = CouplingMap.from_line(4)
pm = generate_preset_pass_manager(coupling_map=coupling_map)
transpiled_circ = pm.run(circ)

print(f"qubits: {circ.num_qubits}")
print(
    f"2q-depth: {transpiled_circ.depth(lambda x: x.operation.num_qubits==2)}"
)
print(f"2q-size: {transpiled_circ.size(lambda x: x.operation.num_qubits==2)}")
print(f"Operator counts: {transpiled_circ.count_ops()}")
transpiled_circ.decompose().draw(
    output="mpl", fold=-1, style="clifford", idle_wires=False
)

qubits: 4
2q-depth: 94
2q-size: 96
Operator counts: OrderedDict({'cx': 45, 'swap': 32, 'u': 24, 'u1': 7, 'u3': 4, 'unitary': 3, 'circuit-335': 1, 'circuit-338': 1, 'circuit-341': 1, 'circuit-344': 1, 'circuit-347': 1, 'circuit-350': 1, 'circuit-353': 1, 'circuit-356': 1, 'circuit-359': 1, 'circuit-362': 1, 'circuit-365': 1, 'circuit-368': 1, 'circuit-371': 1, 'circuit-374': 1, 'circuit-377': 1, 'circuit-380': 1})


<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/c184f6dd-9f80-4487-ac0b-0dd94170b0f0-1.avif" alt="Output of the previous code cell" />

Let's compare these counts with the compiled circuit depth of our manual implementation of the $M_2$ gate.

In [16]:
# Get the M2 operator from our manual construction
M2 = M2mod15()

# Add it to a circuit
circ = QuantumCircuit(4)
circ.compose(M2, inplace=True)
circ = circ.decompose(reps=3)

# Transpile the circuit and get the depth
coupling_map = CouplingMap.from_line(4)
pm = generate_preset_pass_manager(coupling_map=coupling_map)
transpiled_circ = pm.run(circ)

print(f"qubits: {circ.num_qubits}")
print(
    f"2q-depth: {transpiled_circ.depth(lambda x: x.operation.num_qubits==2)}"
)
print(f"2q-size: {transpiled_circ.size(lambda x: x.operation.num_qubits==2)}")
print(f"Operator counts: {transpiled_circ.count_ops()}")
transpiled_circ.draw(
    output="mpl", fold=-1, style="clifford", idle_wires=False
)

qubits: 4
2q-depth: 9
2q-size: 9
Operator counts: OrderedDict({'cx': 9})


<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0235c931-0adb-4972-9fce-32a0341822bf-1.avif" alt="Output of the previous code cell" />

As we can see, the permutation matrix approach resulted in a significantly deep circuit even for a single $M_2$ gate compared to our manual implementation of it. Therefore, we will continue with our previous implementation of the $M_b$ operations.

Now, we are ready to construct the full order finding circuit using our previously defined controlled modular exponentiation operators. In the following code, we also import the [QFT circuit](/docs/api/qiskit/qiskit.circuit.library.QFT) from the Qiskit Circuit library, which uses Hadamard gates on each qubit, a series of controlled-U1 (or Z, depending on the phase) gates, and a layer of swap gates.

In [17]:
# Order finding problem for N = 15 with a = 2
N = 15
a = 2

# Number of qubits
num_target = floor(log(N - 1, 2)) + 1  # for modular exponentiation operators
num_control = 2 * num_target  # for enough precision of estimation

# List of M_b operators in order
k_list = range(num_control)
b_list = [a2kmodN(2, k, 15) for k in k_list]

# Initialize the circuit
control = QuantumRegister(num_control, name="C")
target = QuantumRegister(num_target, name="T")
output = ClassicalRegister(num_control, name="out")
circuit = QuantumCircuit(control, target, output)

# Initialize the target register to the state |1>
circuit.x(num_control)

# Add the Hadamard gates and controlled versions of the
# multiplication gates
for k, qubit in enumerate(control):
    circuit.h(k)
    b = b_list[k]
    if b == 2:
        circuit.compose(
            M2mod15().control(), qubits=[qubit] + list(target), inplace=True
        )
    elif b == 4:
        circuit.compose(
            M4mod15().control(), qubits=[qubit] + list(target), inplace=True
        )
    else:
        continue  # M1 is the identity operator

# Apply the inverse QFT to the control register
circuit.compose(QFT(num_control, inverse=True), qubits=control, inplace=True)

# Measure the control register
circuit.measure(control, output)

circuit.draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0e854aed-c11b-494c-8c80-adeb8eb0e8fe-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/c184f6dd-9f80-4487-ac0b-0dd94170b0f0-1.avif)

Confrontiamo questi conteggi con la profondità del circuito compilato della nostra implementazione manuale della porta $M_2$.

In [ ]:
service = QiskitRuntimeService()
backend = service.backend("ibm_marrakesh")
pm = generate_preset_pass_manager(optimization_level=2, backend=backend)

transpiled_circuit = pm.run(circuit)

print(
    f"2q-depth: {transpiled_circuit.depth(lambda x: x.operation.num_qubits==2)}"
)
print(
    f"2q-size: {transpiled_circuit.size(lambda x: x.operation.num_qubits==2)}"
)
print(f"Operator counts: {transpiled_circuit.count_ops()}")
transpiled_circuit.draw(
    output="mpl", fold=-1, style="clifford", idle_wires=False
)

2q-depth: 187
2q-size: 260
Operator counts: OrderedDict({'sx': 521, 'rz': 354, 'cz': 260, 'measure': 8, 'x': 4})


<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/95925dd5-7ba9-4746-b96e-ba50400fa5ac-1.avif" alt="Output of the previous code cell" />

## Step 3: Execute using Qiskit primitives

First, we discuss what we would theoretically obtain if we ran this circuit on an ideal simulator. Below, we have a set of simulation results of the above circuit using 1024 shots. As we can see, we get an approximately uniform distribution over four bitstrings over the control qubits.

In [19]:
# Obtained from the simulator
counts = {"00000000": 264, "01000000": 268, "10000000": 249, "11000000": 243}

In [20]:
plot_histogram(counts)

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/0d6d2702-02e4-47de-8f7e-0b256657ef0f-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/0e854aed-c11b-494c-8c80-adeb8eb0e8fe-0.avif)

Notate che abbiamo omesso le operazioni di esponenziazione modulare controllate dai qubit di controllo rimanenti perché $M_1$ è l'operatore identità.
Notate che più avanti in questo tutorial, eseguiremo questo circuito sul backend `ibm_marrakesh`. Per fare questo, traspiamo il circuito in accordo a questo backend specifico e riportiamo la profondità del circuito e i conteggi delle porte.

In [21]:
# Rows to be displayed in table
rows = []
# Corresponding phase of each bitstring
measured_phases = []

for output in counts:
    decimal = int(output, 2)  # Convert bitstring to decimal
    phase = decimal / (2**num_control)  # Find corresponding eigenvalue
    measured_phases.append(phase)
    # Add these values to the rows in our table:
    rows.append(
        [
            f"{output}(bin) = {decimal:>3}(dec)",
            f"{decimal}/{2 ** num_control} = {phase:.2f}",
        ]
    )

# Print the rows in a table
headers = ["Register Output", "Phase"]
df = pd.DataFrame(rows, columns=headers)
print(df)

            Register Output           Phase
0  00000000(bin) =   0(dec)    0/256 = 0.00
1  01000000(bin) =  64(dec)   64/256 = 0.25
2  10000000(bin) = 128(dec)  128/256 = 0.50
3  11000000(bin) = 192(dec)  192/256 = 0.75


Recall that the any measured phase corresponds to $\theta = k / r$ where $k$ is sampled uniformly at random from $\{0, 1, \dots, r-1 \}$. Therefore, we can use the continued fractions algorithm to attempt to find $k$ and the order $r$. Python has this functionality built in. We can use the `fractions` module to turn a float into a `Fraction` object, for example:

In [22]:
Fraction(0.666)

Fraction(5998794703657501, 9007199254740992)

![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/95925dd5-7ba9-4746-b96e-ba50400fa5ac-1.avif)


## Passo 3: Eseguire utilizzando i primitive Qiskit
Innanzitutto, discutiamo cosa otterremmo teoricamente se eseguissimo questo circuito su un simulatore ideale. Di seguito, abbiamo una serie di risultati di simulazione del circuito sopra utilizzando 1024 shot. Come possiamo vedere, otteniamo una distribuzione approssimativamente uniforme su quattro bitstring sui qubit di controllo.

In [23]:
# Get fraction that most closely resembles 0.666
# with denominator < 15
Fraction(0.666).limit_denominator(15)

Fraction(2, 3)

This is much nicer. The order (r) must be less than N, so we will set the maximum denominator to be `15`:

In [24]:
# Rows to be displayed in a table
rows = []

for phase in measured_phases:
    frac = Fraction(phase).limit_denominator(15)
    rows.append(
        [phase, f"{frac.numerator}/{frac.denominator}", frac.denominator]
    )

# Print the rows in a table
headers = ["Phase", "Fraction", "Guess for r"]
df = pd.DataFrame(rows, columns=headers)
print(df)

   Phase Fraction  Guess for r
0   0.00      0/1            1
1   0.25      1/4            4
2   0.50      1/2            2
3   0.75      3/4            4


![Output of the previous code cell](../docs/images/tutorials/shors-algorithm/extracted-outputs/0d6d2702-02e4-47de-8f7e-0b256657ef0f-0.avif)

Misurando i qubit di controllo, otteniamo una stima di fase a otto bit dell'operatore $M_a$. Possiamo convertire questa rappresentazione binaria in decimale per trovare la fase misurata. Come possiamo vedere dall'istogramma sopra, sono state misurate quattro bitstring diverse, e ciascuna di esse corrisponde a un valore di fase come segue.

In [ ]:
# Sampler primitive to obtain the probability distribution
sampler = Sampler(backend)

# Turn on dynamical decoupling with sequence XpXm
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XpXm"
# Enable gate twirling
sampler.options.twirling.enable_gates = True

pub = transpiled_circuit
job = sampler.run([pub], shots=1024)

In [25]:
result = job.result()[0]
counts = result.data["out"].get_counts()

In [26]:
plot_histogram(counts, figsize=(35, 5))

<Image src="../docs/images/tutorials/shors-algorithm/extracted-outputs/559d7030-1f67-44e8-afa7-6afc7a334677-0.avif" alt="Output of the previous code cell" />

As we can see, we obtained the same bitstrings with highest counts. Since quantum hardware has noise, there is some leakage to other bitstrings, which we can filter out statistically.

In [27]:
# Dictionary of bitstrings and their counts to keep
counts_keep = {}
# Threshold to filter
threshold = np.max(list(counts.values())) / 2

for key, value in counts.items():
    if value > threshold:
        counts_keep[key] = value

print(counts_keep)

{'00000000': 58, '01000000': 41, '11000000': 42, '10000000': 40}


Poiché questo fornisce frazioni che restituiscono il risultato esattamente (in questo caso, `0.6660000...`), può dare risultati complicati come quello sopra. Possiamo usare il metodo `.limit_denominator()` per ottenere la frazione che assomiglia più da vicino al nostro float, con un denominatore inferiore a un certo valore:

In [28]:
a = 2
N = 15

FACTOR_FOUND = False
num_attempt = 0

while not FACTOR_FOUND:
    print(f"\nATTEMPT {num_attempt}:")
    # Here, we get the bitstring by iterating over outcomes
    # of a previous hardware run with multiple shots.
    # Instead, we can also perform a single-shot measurement
    # here in the loop.
    bitstring = list(counts_keep.keys())[num_attempt]
    num_attempt += 1
    # Find the phase from measurement
    decimal = int(bitstring, 2)
    phase = decimal / (2**num_control)  # phase = k / r
    print(f"Phase: theta = {phase}")

    # Guess the order from phase
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator  # order = r
    print(f"Order of {a} modulo {N} estimated as: r = {r}")

    if phase != 0:
        # Guesses for factors are gcd(a^{r / 2} ± 1, 15)
        if r % 2 == 0:
            x = pow(a, r // 2, N) - 1
            d = gcd(x, N)
            if d > 1:
                FACTOR_FOUND = True
                print(f"*** Non-trivial factor found: {x} ***")


ATTEMPT 0:
Phase: theta = 0.0
Order of 2 modulo 15 estimated as: r = 1

ATTEMPT 1:
Phase: theta = 0.25
Order of 2 modulo 15 estimated as: r = 4
*** Non-trivial factor found: 3 ***


## Discussion

### Related work
In this section, we discuss other milestone work that has demonstrated Shor's algorithm on real hardware.

The seminal work [[3]](#references) from IBM&reg; demonstrated Shor's algorithm for the first time, factoring the number 15 into its prime factors 3 and 5 using a seven-qubit nuclear magnetic resonance (NMR) quantum computer. Another experiment [[4]](#references) factored 15 using photonic qubits. By employing a single qubit recycled multiple times and encoding the work register in higher-dimensional states, the researchers reduced the required number of qubits to one-third of that in the standard protocol, utilizing a two-photon compiled algorithm. A significant paper in the demonstration of Shor's algorithm is [[5]](#references), which uses Kitaev's iterative phase estimation [[8]](#references) technique to reduce the qubit requirement of the algorithm. Authors used seven control qubits and four cache qubits, together with the implementation of modular multipliers. This implementation, however, requires mid-circuit measurements with feed-forward operations and qubit recycling with reset operations. This demonstration was done on an ion-trap quantum computer.

More recent work [[6]](#references) focused on factoring 15, 21, and 35 on IBM Quantum&reg; hardware. Similar to previous work, researchers used a compiled version of the algorithm that employed a semi-classical quantum Fourier transform as proposed by Kitaev to minimize the number of physical qubits and gates. A most recent work [[7]](#references) also performed a proof-of-concept demonstration for factoring the integer 21. This demonstration also involved the use of a compiled version of the quantum phase estimation routine, and built upon the previous demonstration by [[4]](#references). Authors went beyond this work by using a configuration of approximate Toffoli gates with residual phase shifts. The algorithm was implemented on IBM quantum processors using only five qubits, and the presence of entanglement between the control and register qubits was verified successfully.

### Scaling of the algorithm

We note that RSA encryption typically involves key sizes on the order of 2048 to 4096 bits. Attempting to factor a 2048-bit number with Shor's algorithm will result in a quantum circuit with millions of qubits, including the error correction overhead and a circuit depth on the order of a billion, which is beyond the limits of current quantum hardware to execute. Therefore, Shor's algorithm will require either optimized circuit construction methods or robust quantum error correction to be practically viable for breaking modern cryptographic systems. We refer you to [[9]](#references) for a more detailed discussion on resource estimation for Shor's algorithm.

## Challenge

Congratulations for finishing the tutorial! Now is a great time to test your understanding. Could you try to construct the circuit for factoring 21? You can select an $a$ of your own choice. You will need to decide on the bit accuracy of the algorithm to choose the number of qubits, and you will need to design the modular exponentiation operators $M_a$. We encourage you to try this out yourself, and then read about the methodologies shown in Fig. 9 of [[6]](#references) and Fig. 2 of [[7]](#references).

In [ ]:
def M_a_mod21():
    """
    M_a (mod 21)
    """

    # Your code here
    pass

Questo è molto più chiaro. L'ordine (r) deve essere inferiore a N, quindi imposteremo il denominatore massimo su `15`: